In [1]:
# finding available datasets

import openpolicedata as opd
import pandas as pd

pd.options.display.max_colwidth = 0
datasets = opd.datasets.query()

# this will count, sort, and concatenate grouped columns with .apply() later
def groupInfoWithCount(df):
  df_counted = df.value_counts().reset_index()
  df_counted.columns = ['Value', 'Count']
  def doCount(row):
    return f"{row['Value']} ({row['Count']})"
  df_counted['Value_Count'] = df_counted.apply(doCount, axis=1)
  return ', '.join(df_counted['Value_Count'])

grouped = datasets.groupby('State')
# making SourceName Info column
sourcename_info = (
  grouped['SourceName']
  .apply(groupInfoWithCount)
  .reset_index(name='SourceNames')
)
# making TableType Info column
tabletype_info = (
  grouped['TableType']
  .apply(groupInfoWithCount)
  .reset_index(name='TableTypes')
)
# making Year Info column
year_info = (
  datasets[['State', 'Year']]
  .astype(str)
  .groupby('State')['Year']
  .apply(groupInfoWithCount)
  .reset_index(name='Years')
)
# merging and printing them all
info = (
  sourcename_info
  .merge(tabletype_info, on='State')
  .merge(year_info, on='State')
)
caption = '<h2><b>Datasets Available by State (and how many Datasets there are for each SourceName/TableType/Year)</b></h2>'
info.style.hide(axis='index').set_caption(caption)


State,SourceNames,TableTypes,Years
Arizona,"Tucson (13), Phoenix (11), Gilbert (3), Mesa (1), State Patrol (1)","CALLS FOR SERVICE (17), STOPS (3), EMPLOYEE (2), OFFICER-INVOLVED SHOOTINGS - CIVILIANS (1), OFFICER-INVOLVED SHOOTINGS - INCIDENTS (1), OFFICER-INVOLVED SHOOTINGS - OFFICERS (1), ARRESTS (1), TRAFFIC CITATIONS (1), OFFICER-INVOLVED SHOOTINGS (1), USE OF FORCE (1)","MULTI (10), NONE (3), 2016 (2), 2017 (2), 2018 (2), 2019 (2), 2020 (2), 2021 (2), 2012 (1), 2013 (1), 2014 (1), 2015 (1)"
Arkansas,Little Rock (2),"TRAFFIC STOPS (1), CALLS FOR SERVICE (1)",MULTI (2)
California,"Sacramento (20), Los Angeles (14), San Diego (12), San Jose (12), California (12), Oakland (11), Santa Rosa (9), Menlo Park (7), Richmond (7), Los Angeles County (3), San Francisco (2), Stockton (2), San Bernardino (1), Long Beach (1), Santa Ana (1), Anaheim (1), Bakersfield (1), Berkeley (1), Corona (1), State Patrol (1)","CALLS FOR SERVICE (50), USE OF FORCE (12), CITATIONS (10), STOPS (8), TRAFFIC STOPS (7), USE OF FORCE - CIVILIANS/OFFICERS (6), USE OF FORCE - INCIDENTS (6), INCIDENTS (6), COMPLAINTS (4), VEHICLE PURSUITS (2), OFFICER-INVOLVED SHOOTINGS - INCIDENTS (1), OFFICER-INVOLVED SHOOTINGS - OFFICERS (1), OFFICER-INVOLVED SHOOTINGS - CIVILIANS (1), EMPLOYEE (1), OFFICER-INVOLVED SHOOTINGS (1), CRASHES - INCIDENTS (1), CRASHES - CIVILIANS (1), CRASHES (1)","MULTI (34), 2019 (12), 2018 (11), 2017 (10), 2016 (10), 2021 (9), 2020 (9), 2015 (8), 2014 (6), 2012 (3), 2013 (3), 2011 (2), 2010 (1), 2022 (1)"
Colorado,"Denver (2), Aurora (1), State Patrol (1)","STOPS (2), OFFICER-INVOLVED SHOOTINGS (1), TRAFFIC STOPS (1)",MULTI (4)
Connecticut,"Norwich (3), Hartford (1), Connecticut (1)","USE OF FORCE (3), TRAFFIC STOPS (2)","MULTI (2), 2017 (1), 2018 (1), 2019 (1)"
Florida,"Fort Lauderdale (3), Saint Petersburg (2), State Patrol (1), Tampa (1), Orlando (1), Jacksonville (1)","TRAFFIC STOPS (3), OFFICER-INVOLVED SHOOTINGS (2), CALLS FOR SERVICE (2), EMPLOYEE (1), TRAFFIC CITATIONS (1)","MULTI (8), NONE (1)"
Georgia,"Johns Creek (3), State Patrol (1)","TRAFFIC STOPS (1), CALLS FOR SERVICE (1), CITATIONS (1), CRASHES (1)",MULTI (4)
Idaho,"Idaho Falls (1), Boise (1)","STOPS (1), CALLS FOR SERVICE (1)",MULTI (2)
Illinois,"Chicago (3), Illinois (1)","STOPS (1), TRAFFIC STOPS (1), ARRESTS (1), COMPLAINTS (1)",MULTI (4)
Indiana,"St. John (64), Bloomington (7), Indianapolis (3), South Bend (2), Fort Wayne (1)","CALLS FOR SERVICE (17), ARRESTS (16), TRAFFIC WARNINGS (16), TRAFFIC CITATIONS (16), COMPLAINTS (3), OFFICER-INVOLVED SHOOTINGS (2), EMPLOYEE (2), USE OF FORCE (2), TRAFFIC STOPS (1), VEHICLE PURSUITS (1), CRASHES (1)","MULTI (11), 2017 (5), 2016 (5), 2013 (4), 2020 (4), 2019 (4), 2018 (4), 2015 (4), 2014 (4), 2011 (4), 2012 (4), 2010 (4), 2009 (4), 2008 (4), 2007 (4), 2006 (4), 2021 (4)"


In [8]:
# specifying datasets to load

from typing import Tuple, List
import openpolicedata as opd

# Necessary - these must be chosen
USE_EXAMPLE: bool = True
SOURCE_NAME: str = 'Detroit'

# Optional
LIMIT: int = None
LIMIT_DEFAULT = None

STATE: str = None
STATE_DEFAULT = None

YEAR: int or Tuple(int, int) = None
YEAR_DEFAULT = None

TABLETYPES: str or List[str] = None
TABLETYPES_DEFAULT = None

COUNT_AND_SORT: bool = None
COUNT_AND_SORT_DEFAULT = False

# Examples
SOURCE_NAME_EXAMPLE = "Detroit"
LIMIT_EXAMPLE = 50
YEAR_EXAMPLE = (2020, 2022)
TABLETYPES_EXAMPLE = ['CALLS FOR SERVICE', 'INCIDENTS']

''' Variables 

The following variables are necessary, and there are no defaults.

  USE_EXAMPLE: Whether to use example values or not.
    The example takes less than a minute to load via OPD API.

  CHOSEN_SOURCE_NAME: Choose a SourceName.
    See above cell for options in column 'SourceNames'.

The following variables are option, and there are defaults that can be used.

  LIMIT: Choose how many rows to load from the API.
    Default will load all available rows.
    Default: None

  CHOSEN_STATE: Choose a State.
    Only needed if CHOSEN_SOURCE_NAME is 'State Patrol'.
    Default: None

  CHOSEN_YEAR: Choose what year(s) to load.
    A tuple of two years will load that range, inclusive.
    'Multi' is a valid year selection for some sources.
    Default will load all available years.
    Default: None

  CHOSEN_TABLETYPE: Choose what table type(s) to load.
    A list of valid table types will load each of them.
    Default will load all available table types.
    Default: None

  COUNT_AND_SORT: Choose if table type(s) will download smallest-first.
    Takes longer (extra API calls), but can help gauge loading speeds.
    Default: True
  
'''

source = SOURCE_NAME if not USE_EXAMPLE else SOURCE_NAME_EXAMPLE
state = STATE or STATE_DEFAULT
year = YEAR or YEAR_DEFAULT if not USE_EXAMPLE else YEAR_EXAMPLE
limit = LIMIT or LIMIT_DEFAULT if not USE_EXAMPLE else LIMIT_EXAMPLE
src = opd.Source(source_name=source, state=state)
if USE_EXAMPLE:
  table_types = TABLETYPES_EXAMPLE
elif not TABLETYPES:
  table_types = src.get_tables_types()
elif isinstance(TABLETYPES, list):
  table_types = [*TABLETYPES]
else:
  table_types = [TABLETYPES]
if COUNT_AND_SORT is None:
  COUNT_AND_SORT = COUNT_AND_SORT_DEFAULT
if COUNT_AND_SORT:
  def doSortAndTime(table_type):
    count = src.get_count(table_type=table_type, year=year)
    display(f"{table_type} contains {count} rows.")
    return count
  table_types = sorted(table_types, key=doSortAndTime)

display(f"Ready to load {limit or 'all'} rows from {source} tables {table_types} for {year or 'all available years'}.")

"Ready to load 50 rows from Detroit tables ['CALLS FOR SERVICE', 'INCIDENTS'] for (2020, 2022)."

In [9]:
# loading datasets, storing them offline

import os
import pandas as pd
import zipfile as zf
import pickle

LOAD_FROM_OFFLINE = True
LOAD_FROM_ONLINE = True
SAVE_TO_OFFLINE = True
HIDE_DISPLAY_INFO = False

display(f"Retrieving datasets {table_types}...")

dataAndTables = {table_type: {} for table_type in table_types}
expected_files = [f"{table_type}.csv" for table_type in table_types] + \
                 [f"{table_type}.pickle" for table_type in table_types]

def displayInfo(table_type: str = None, dataAndTables: dict[dict] = None):
  if HIDE_DISPLAY_INFO: return
  if not dataAndTables.get(table_type): return
  if not dataAndTables[table_type].get('tableIsLoaded'): return
  if not dataAndTables[table_type].get('data'): return
  table = dataAndTables[table_type]['table']
  df_size = table.memory_usage().sum() * 1.024e-6 # mem in MB
  description = dataAndTables[table_type]['data'].description
  display(f"{table_type} ({'%.2f' % df_size} MB): {description}")
  # if not isinstance(dataAndTables[table_type].get('table'), pd.DataFrame): return
  # display(f"{table_type} ({'%.2f' % df_size} MB): {description}")
    
# check if offline data is available, then load it
year_str = 'All' if year == None \
  else '-'.join([str(y) for y in year]) if isinstance(year, tuple) \
  else str(year)
table_types_str = '' if table_types == None \
  else '-'.join([t[:3] for t in table_types]) if isinstance(table_types, list) \
  else table_types
zipfile_parts = [
  'data',
  source,
  table_types_str,
  year_str,
  limit,
]
zipfile = '_'.join([str(p) for p in zipfile_parts if p])
zipfile += '.zip'

directory = os.getcwd()
files_in_data_directory = os.listdir(f"{directory}/data")
zipfile_path = f"{directory}/data/{zipfile}"
temp_folder = f"{directory}/data/{zipfile.split('.')[0]}"
loaded_files = []
if LOAD_FROM_OFFLINE and zipfile in files_in_data_directory:
  display(f"Loading from offline - {zipfile}")
  if os.path.exists(temp_folder):
    [os.remove(f"{temp_folder}/{file}") for file in os.listdir(temp_folder)]
    os.rmdir(temp_folder)
  with zf.ZipFile(zipfile_path, 'r', zf.ZIP_DEFLATED) as zip_file:
    available_files = [file.filename for file in zip_file.infolist()]
    # available_files = [name.split(
    #   '/')[-1] for name in available_files if name.endswith('csv') or name.endswith('pickle')]  
    common_files = list(set(expected_files) & set(available_files))
    # extract pickles first to properly run displayInfo
    common_files.sort(key=lambda x: x.endswith('.pickle'), reverse=True)
    
    # display(expected_files, available_files, common_files)
    
    for file in common_files:
      zip_file.extract(file, temp_folder)
      tempfile = f"{temp_folder}/{file}"
      table_type = file.split('.')[0]
      if file.endswith('.pickle'):
        with open(tempfile, 'rb') as f:
          data = pickle.load(f)
          dataAndTables[table_type]['data'] = data
          dataAndTables[table_type]['tableIsLoaded'] = True
      if file.endswith('.csv'):
        t = pd.read_csv(tempfile, low_memory=False)
        dataAndTables[table_type]['table'] = t
        displayInfo(table_type, dataAndTables)
      os.remove(tempfile)
      loaded_files.append(file)
    if os.path.exists(temp_folder): os.rmdir(temp_folder)
elif LOAD_FROM_OFFLINE and zipfile not in files_in_data_directory:
  display(f"Cannot load from offline - {zipfile} not found")

# load the remaining datasets using opd api
remaining_files = list(set(expected_files) ^ set(loaded_files))
remaining_table_types = list({file.split('.')[0] for file in remaining_files})
if LOAD_FROM_ONLINE and any(remaining_files):
  display(f"Loading from online: {remaining_table_types}")
  os.mkdir(temp_folder)
  for table_type in remaining_table_types:
    yearparam = [*year] if isinstance(year, tuple) else year
    data = src.load_from_url(table_type=table_type, year=yearparam, nrows=limit)
    t = data.table
    # save df to .csv
    csv_filename = f"{temp_folder}/{table_type}.csv"
    t.to_csv(csv_filename, index=True)
    dataAndTables[table_type]['table'] = t
    # save data to .pickle
    data.table = csv_filename
    dataAndTables[table_type]['data'] = data
    pickle_filename = f"{temp_folder}/{table_type}.pickle"
    with open(pickle_filename, 'wb') as pickle_file:
      pickle.dump(data, pickle_file)
    displayInfo(table_type, dataAndTables)
  
# save .csv and .pickle files to .zip
if SAVE_TO_OFFLINE and any(remaining_files):
  with zf.ZipFile(zipfile_path, 'w', zf.ZIP_DEFLATED) as zip_file:
    files = os.listdir(temp_folder)
    for file in files:
      file_path = os.path.join(temp_folder, file)
      zip_file.write(file_path, os.path.basename(file_path))
      # os.remove(file_path)
      # os.listdir(temp_folder)
  # os.remove(temp_folder)
  zip_size = os.path.getsize(zipfile_path) * 1.024e-6
  display(f"Saved datasets to disk as {zipfile_path} ({'%.2f' % zip_size} MB)")


"Retrieving datasets ['CALLS FOR SERVICE', 'INCIDENTS']..."

'Cannot load from offline - data_Detroit_CAL-INC_2020-2022_50.zip not found'

"Loading from online: ['CALLS FOR SERVICE', 'INCIDENTS']"

'Saved datasets to disk as /Users/alex/Desktop/Code/explore_police_data/data/data_Detroit_CAL-INC_2020-2022_50.zip (0.01 MB)'

In [10]:
# quick check of dataframe sizes

every_column = set()
for table_type in dataAndTables.keys():
  t = dataAndTables[table_type]['table']
  cols = t.columns
  every_column = every_column.union(set(cols))
  display(f"{table_type} has {t.size:,d} rows and {t.shape[1]} columns: {list(cols)}")
  # matches = ['time','date']
  # matched_cols = cols[cols.str.contains('|'.join(matches), case=False)]
  # display(f"{table_type} columns with {matches}: {list(matched_cols)}")

"CALLS FOR SERVICE has 1,300 rows and 26 columns: ['incident_id', 'agency', 'incident_address', 'zip_code', 'priority', 'callcode', 'calldescription', 'category', 'call_timestamp', 'precinct_sca', 'respondingunit', 'officerinitiated', 'intaketime', 'dispatchtime', 'traveltime', 'totalresponsetime', 'time_on_scene', 'totaltime', 'neighborhood', 'block_id', 'council_district', 'longitude', 'latitude', 'shape', 'ObjectId', 'geolocation']"

"INCIDENTS has 1,200 rows and 24 columns: ['crime_id', 'report_number', 'address', 'offense_description', 'offense_category', 'state_offense_code', 'arrest_charge', 'charge_description', 'incident_timestamp', 'incident_time', 'day_of_week', 'hour_of_day', 'year', 'scout_car_area', 'precinct', 'block_id', 'neighborhood', 'council_district', 'zip_code', 'longitude', 'latitude', 'ibr_date', 'oid', 'geolocation']"

In [11]:
# compare unique values across all columns

tables = {t: dataAndTables[t]['table'] for t in dataAndTables.keys()}

all_columns = []
for table_type, t in tables.items():
  all_columns += list(dataAndTables[table_type]['table'].columns)
unique_columns = sorted(set(all_columns))

column_data = {}
for table_type, t in tables.items():
  column_data[table_type] = []
for column in unique_columns:
  for table_type, t in tables.items():
    column_data[table_type].append(column)
df_comparingColumns = pd.DataFrame(column_data, index=unique_columns)

all_dataframes = [dataAndTables[table_type]['table']
                  for table_type in dataAndTables.keys()]
all_data = pd.concat(all_dataframes)
for column in unique_columns:
  unique_values = all_data[column].nunique()
  df_comparingColumns.loc[column, 'ALL'] = unique_values

for table_type, t in tables.items():
  for column in unique_columns:
    if column in t.columns:
      unique_values = t[column].nunique()
    else:
      unique_values = 0
    df_comparingColumns.loc[column, table_type] = unique_values

compareeverything = (df_comparingColumns
                     .reindex(sorted(df_comparingColumns.columns), axis=1)
                     .sort_values(by='ALL', ascending=False).astype(int)
                    )

filter_ranges = compareeverything.describe().astype(int)

display(filter_ranges, compareeverything)
# count_stack = filter_ranges.loc['25%', 'ALL']
# count_filter = filter_ranges.loc['50%', 'ALL']
# display(f"25% mark: {count_stack}, 50% mark: {count_filter}", compareeverything)

TypeError: unhashable type: 'dict'

In [ ]:
# stacked bar chart to expolore the data

import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, IntSlider, interact

tables = {t: dataAndTables[t]['table'] for t in dataAndTables.keys()}
table_types = list(tables.keys())

def generateStackedBarChart(table_type,
                            filter1,
                            filter1_value,
                            filter2,
                            num_bars,
                            stack):
  table = tables[table_type]
  filtered = table[table[filter1] == filter1_value]
  grouped = (filtered
                   #  .dropna(subset=filter2)
                   .groupby([filter2, stack])
                   .size()
                   .unstack(fill_value=0))
  sorted_groups = (grouped.sum(axis=1)
                   .sort_values(ascending=False)
                   .head(num_bars))
  grouped = grouped.loc[sorted_groups.index]
  ax = grouped.plot(kind='bar', stacked=True, figsize=(10, 6))
  ax.set_xlabel(filter2)
  ax.set_ylabel('Count')
  ax.set_title(f'{filter1}={filter1_value}, {filter2} - {stack}')
  plt.show()

def moveToFront(columns, elements):
  cloned_list = [*columns]
  if type(elements) is not list:
    cloned_elements = [elements]
  else:
    cloned_elements = [*elements]
  cloned_elements.reverse()
  for e in cloned_elements:
    if e in cloned_list:
      cloned_list.insert(0, e)
      cloned_list.remove(e)
  # display(f"some_list: {columns[5:]}, cloned_list: {cloned_list[5:]}")
  return cloned_list

def get_filter_columns(table_type):
  default_front_columns = ['neighborhood', 'calldescription']
  t = tables[table_type]
  columns = [c for c in t.columns if 16 < t[c].nunique() < 512]
  # display(f"get_filter_columns: {columns[5:]}")
  columns = moveToFront(columns=columns, elements=default_front_columns)
  return columns

def get_stack_columns(table_type):
  default_front_columns = ['officerinitiated']
  t = tables[table_type]
  columns = [c for c in t.columns if t[c].nunique() <= 16]
  # display(f"get_stack_columns: {columns[5:]}")
  columns = moveToFront(columns=columns, elements=default_front_columns)
  return columns

table_type_widget = Dropdown(options=table_types, value=table_types[0])
filter_columns = get_filter_columns(table_type_widget.value)
# display(f"filter_columns: {filter_columns}")
filter1_widget = Dropdown(options=filter_columns, value=filter_columns[0])
filter1_values = tables[table_type_widget.value][filter1_widget.value].unique()
filter1_values_widget = Dropdown(options=filter1_values, value=filter1_values[0])
filter2_widget = Dropdown(options=filter_columns, value=filter_columns[1])
num_bars_widget = IntSlider(min=1, max=32, value=10)
stack_columns = get_stack_columns(table_type_widget.value)
stack_widget = Dropdown(options=stack_columns, value=stack_columns[0])

def table_changed(change):
  table = tables[table_type_widget.value]
  filter_columns = get_filter_columns(table_type_widget.value)
  filter1_widget.options = filter_columns
  filter1_values_widget.options = table[filter1_widget.value].unique()
  filter2_widget.options = filter_columns[1:]
  stack_widget.options = get_stack_columns(table_type_widget.value)
  
def filter1_changed(change):
  table = tables[table_type_widget.value]
  values = table[filter1_widget.value].unique()
  filter1_values_widget.options = values
  if filter1_widget.value == filter2_widget.value:
    filter2_widget.value == filter2_widget.options[0]
  if filter1_widget.value == filter2_widget.value:
    filter2_widget.value == filter2_widget.options[1]
  if filter1_widget.value == filter2_widget.value:
    raise
  
table_type_widget.observe(table_changed, 'value')
filter1_widget.observe(filter1_changed, 'value')

interact(generateStackedBarChart,
         table_type=table_type_widget,
         filter1=filter1_widget,
         filter1_value=filter1_values_widget,
         filter2=filter2_widget,
         num_bars=num_bars_widget,
         stack=stack_widget)


TypeError: unhashable type: 'dict'

In [ ]:
# trying to tie in some census data, found this api
# https://github.com/datamade/census

import os
from census import Census
from us import states

CENSUS_API_KEY = os.environ.get('CENCUS_API_KEY')

c = Census(CENSUS_API_KEY)

# test mostly from census readme
# census_data = c.acs5.get(('NAME', 'B25034_010E'), {
census_data=c.acs5.get(('NAME', 'B19013_001E'), {
  'for': 'state:{}'.format(states.MI.fips)
})

display(census_data)

census_tables = c.acs5.tables()
tables_df = pd.DataFrame(census_tables)
match_str = 'SEX BY AGE'
# display(tables_df[tables_df['description'].str.contains(match_str)])
tables_df

census_data = c.acs5.get(('NAME', 'B19013_001E'), {
                         'for': 'zip code tabulation area:*'})
income_df = pd.DataFrame(census_data)
income_df.rename(
    columns={'B19013_001E': 'median_income', 'NAME': 'geo'}, inplace=True)
income_df['zip_code'] = income_df['geo'].str[-5:]
income_df.drop(columns=['geo'], inplace=True)
income_df


[{'NAME': 'Michigan', 'B19013_001E': 59234.0, 'state': '26'}]

,median_income,zip code tabulation area,zip_code
0,30985.0,29590,29590
1,54450.0,93306,93306
2,39625.0,93660,93660
3,93264.0,93110,93110
4,42983.0,93212,93212
...,...,...,...
33115,51667.0,16623,16623
33116,45000.0,16627,16627
33117,51500.0,16634,16634
33118,55982.0,16640,16640
